# 동의어사전 - 한글/영어 키보드 기준 변환

2021-01-28  
&nbsp;  
주어진 키워드를..
- 한글로 적혀진 것 > 키보드 기준 영어로  
- 영어로 적혀진 것 > 키보드 기준 한글로  

변환하는 작업을 하였다.  
유입되는 검색어 중에 한/영 전환이 필요한 키워드를 찾기 위한 것이다.

In [1]:
import pandas as pd
import re
import warnings
warnings.filterwarnings(action='ignore')

파일불러오기

In [2]:
sd = pd.read_excel('data.xlsx')

## 한글 → 영어로 변환

1. 자소분리  
사과 → ㅅㅏㄱㅘ

In [3]:
from jamo import h2j, j2hcj

sd['자소분리'] = sd['keyword'].apply(lambda x: j2hcj(h2j(x)) if type(x) == str else x)

2. 한글-영문 키보드입력에 맞추어 변환

In [4]:
keyboard = {'ㄱ':'r', 'ㄲ':'R', 'ㄴ':'s', 'ㄷ':'e', 'ㄸ':'E', 'ㄹ':'f', 'ㅁ':'a', 'ㅂ':'q', 'ㅃ':'Q', 'ㅅ':'t', 'ㅆ':'T',
           'ㅇ':'d', 'ㅈ':'w', 'ㅉ':'W', 'ㅊ':'c', 'ㅋ':'z', 'ㅌ':'x', 'ㅍ':'v', 'ㅎ':'g',
           'ㅏ':'k', 'ㅐ':'o', 'ㅑ':'i', 'ㅒ':'O', 'ㅓ':'j', 'ㅔ':'p', 'ㅕ':'u', 'ㅖ':'P', 'ㅗ':'h', 'ㅘ':'hk', 'ㅙ':'ho', 'ㅚ':'hl',
           'ㅛ':'y', 'ㅜ':'n', 'ㅝ':'nj', 'ㅞ':'np', 'ㅟ':'nl', 'ㅠ':'b',  'ㅡ':'m', 'ㅢ':'ml', 'ㅣ':'l',
           'ㄳ':'rt', 'ㄵ':'sw', 'ㄶ':'sg', 'ㄺ':'fr', 'ㄻ':'fa', 'ㄼ':'fq', 'ㄽ':'ft', 'ㄾ':'fx', 'ㄿ':'fv', 'ㅀ':'fg', 'ㅄ':'qt'}

def keng(kwd):
    return ''.join([keyboard[w] if w in keyboard else w for w in kwd])

In [5]:
sd['keyword_Eng'] = sd['자소분리'].apply(lambda x: keng(x) if type(x) == str else x)

결과 예시

In [6]:
sd[['keyword', '자소분리', 'keyword_Eng']].sample(3)

,keyword,자소분리,keyword_Eng
166748,도자기찬기,ㄷㅗㅈㅏㄱㅣㅊㅏㄴㄱㅣ,ehwkrlcksrl
88359,감자 핫도그,ㄱㅏㅁㅈㅏ ㅎㅏㅅㄷㅗㄱㅡ,rkawk gktehrm
58654,프라다장지갑,ㅍㅡㄹㅏㄷㅏㅈㅏㅇㅈㅣㄱㅏㅂ,vmfkekwkdwlrkq


## 영어 → 한글로 변환

1. 영문-한글 키보드 딕셔너리 생성

In [7]:
ek_dict = {'r':'ㄱ', 'R':'ㄲ', 's':'ㄴ', 'e':'ㄷ', 'E':'ㄸ', 'f':'ㄹ', 'a':'ㅁ', 'q':'ㅂ', 'Q':'ㅃ', 't':'ㅅ', 'T':'ㅆ',
           'd':'ㅇ', 'w':'ㅈ', 'W':'ㅉ', 'c':'ㅊ', 'z':'ㅋ', 'x':'ㅌ', 'v':'ㅍ', 'g':'ㅎ',
           
           'k':'ㅏ', 'o':'ㅐ', 'i':'ㅑ', 'O':'ㅒ', 'j':'ㅓ', 'p':'ㅔ', 'u':'ㅕ', 'P':'ㅖ', 'h':'ㅗ', 'hk':'ㅘ', 'ho':'ㅙ', 'hl':'ㅚ',
           'y':'ㅛ', 'n':'ㅜ', 'nj':'ㅝ', 'np':'ㅞ', 'nl':'ㅟ', 'b':'ㅠ',  'm':'ㅡ', 'ml':'ㅢ', 'l':'ㅣ',
           
           'rt':'ㄳ', 'sw':'ㄵ', 'sg':'ㄶ', 'fr':'ㄺ', 'fa':'ㄻ', 'fq':'ㄼ', 'ft':'ㄽ', 'fx':'ㄾ', 'fv':'ㄿ', 'fg':'ㅀ', 'qt':'ㅄ'}

2. 영문을 한글로 변환 & 한글 자모 결합

In [8]:
#! pip install hangul-utils
from hangul_utils import join_jamos

def engkor(text):
    temp = ''
    i = 0
    while i < len(text):
        k2 = text[i:i+2]   # ㄻ ㅞ 같은 글자는 알파벳이 두개가 이어진 것으로 구성되어 있으므로 두 글자를 불러온다.
        if k2 in ek_dict:   
            temp+=ek_dict[k2]
            j=2   # 있으면 다음 글자는 건너뛰어야해서 jump 2
        else:   # 연속된 두글자가 없었으므로 한 글자만 확인한다. 
            k1 = text[i]
            if k1 in ek_dict:
                temp+=ek_dict[k1]
            else:
                temp+=k1
            j=1   # 이 경우엔 바로 다음 글자로 넘어가니까 jump 1
        i+=j   # 위에서 받아온 jump를 i에 더해주어 while loop를 진행한다.
    
    # temp에는 'ㅅㅏㄱㅘ' 처럼 자모가 분리되어있다. 
    return join_jamos(temp)   #자모 합치기

In [9]:
engkor('tkrhk')

'사과'

In [10]:
sd['engkor'] = sd['keyword'].apply(lambda x: engkor(x) if type(x)==str else '')

3. 영문으로만 이루어진 키워드 찾기  
영어가 포함된 한글 키워드도 어중간하게 변화되어버렸다.  
해당 키워드들을 구분짓기 위해 영문으로만 구성된 키워드를 표시해둔다.

In [11]:
import re

# 영문으로만 이루어진 키워드 찾기
com = re.compile('^[a-zA-Z0-9]+$')
sd['eng'] = sd['keyword'].apply(lambda x : 'eng' if type(x) == str and com.search(x)!= None else 'not')